# Helioswarm configuration plotting raw code

In [ ]:
import plotly.graph_objects as go
import numpy as np
import math as m 
import re

In [ ]:
def satellites_coordinates(file_name):
    """
    Parameters
    ----------
    file_name : string
        Name of the file containing the positions of the wanted satellite.
    Returns
    -------
    array
        The array containing the values of x, y and z.
    """
    with open(file_name, "r") as file:
        lines = file.readlines()

    extracted_lines = lines[6:]  # Ignore the lines that don't have values

    list_values = []

    for line in extracted_lines:
        line = line.strip()  # Remove blank spaces and other symbols of the new line 

        values = re.findall(r"[-+]?\d+\.\d+", line)

        # Extraction of the x, y, z values
        x = float(values[1])
        y = float(values[2])
        z = float(values[3])

        list_values.append([x, y, z])  # Add the x, y, z coordinates at the list of values

    # Create a 3 rows table with the values of x , y and z
    final_extracted_lines = np.array(list_values)

    return final_extracted_lines


In [ ]:
def retrieve_time(file_name):
    with open(file_name, "r") as file:
        lines = file.readlines()

    extracted_lines = lines[6:]  # Ignore the lines that don't have values

    list_values = []
    
    for line in extracted_lines:
        line = line.strip()  # Remove blank spaces and other symbols of the new line 
        value = re.findall(r"\d{1,2} [A-Za-z]{3} \d{4} \d{2}:\d{2}:\d{2}.\d{3}", line)
        list_values.append(value)
    
    return list_values

In [ ]:
def combinations(L, n):
    """
    Parameters
    ----------
    L : list
        List of the elements we are working with, e.g., [0, 1, 2, 3, ..., 8]
    n : int
        Number of elements in a combination.

    Returns
    -------
    list
        The list of the combinations of n elements of L.
    """
    if n == 0:
        return [[]]
    else:
        comb = []
        for i in range(len(L)):
            element = L[i]
            rest = L[i+1:]
            for c in combinations(rest, n-1):
                comb.append([element]+c)
        return comb

In [ ]:
def barycenter_t(list_sat,list_index,time_index):
    """
    Parameters
    ----------
    list_sat : list
        List of the position of each satellite, over time.
    list_index : list
        List of the indexes of the studied satellites, e.g., [0, 1, 2, 3, ..., 8]
    time_index : int
        Time.

    Returns
    -------
    bar_t : Array of size (3,)
        Barycenter of the satellites at time t.
        
    """
    bar_t = np.zeros(3, dtype=float)
    for i in list_index:
        bar_t += list_sat[i][time_index][...]
    bar_t /= len(list_index)
    return bar_t

In [ ]:
def barycenter(list_sat,list_index):
    """
    Computes the coordinates of the barycenter, for all time.

    Parameters
    ----------
    list_sat : list
        List of the position of each satellite, over time.
    list_index : list
        List of the indexes of the studied satellites.

    Returns
    -------
    bar : list
        Barycenter of the satellites for all t.

    """
    bar = []
    max_time_index = len(satellites[0][...])
    for i in range(max_time_index):
        bar.append(barycenter_t(list_sat,list_index,i))
    return bar

In [ ]:
def volumetricTensor_t(list_sat,list_index,time_index):
    """
    Computes the coefficients of the volumetric tensor, at time t.

    Parameters
    ----------
    list_sat : list
        List of the positions of each satellite, over time.
    list_index : list
        List of the indexes of the studied satellites.
    time_index : int
        Time.

    Returns
    -------
    vt_t : Array of size (3,3)
        Volumetric tensor, at time time_index.

    """
    vt_t = np.zeros((3,3))
    bar_t = barycenter_t(list_sat,list_index,time_index)
    for j in range(3):
        for k in range(j,3):
            r_jk = 0
            for i in list_index:
                r_jk += (list_sat[i][time_index][j] - bar_t[j])*(list_sat[i][time_index][k] - bar_t[k])
            r_jk /= len(list_index)
            vt_t[j,k] = r_jk
            if (j != k):
                vt_t[k,j] = r_jk
    return vt_t

In [ ]:
def volumetricTensor(list_sat,list_index):
    """
    Computes the coefficients of the volumetric tensor, for all time.

    Parameters
    ----------
    list_sat : list
        List of the positions of each satellite, over time.
    list_index : list
        List of the indexes of the studied satellites.

    Returns
    -------
    vt : Array of size (max_time_index,3,3)
        Volumetric tensor of the satellites for all time.

    """
    max_time_index = len(satellites[0][...])
    vt = np.zeros((max_time_index,3,3))
    for t in range(max_time_index):
        vt[t] = volumetricTensor_t(list_sat,list_index,time_index)
    return vt

In [ ]:
def polyhedronProperties_t(list_sat,list_index,time_index):
    """
    Computes L,E and P, to caracterize the tetrahedron whose vertices are the satellites.

    Parameters
    ----------
    list_sat : list
        List of the positions of each satellite, over time.
    list_index : list
        List of the indexes of the studied satellites.
    time_index : int
        Time.

    Returns
    -------
    L : float
        Caracteristic size of the tetrahedron.
    E : float
        Elongation of the tetrahedron.
    P : float
        Planarity of the tetrahedron.

    """
    if len(list_index) ==4:
        vt_t = volumetricTensor_t(list_sat,list_index,time_index)
        w, v = np.linalg.eig(vt_t)
        w.sort()
        a = np.sqrt(w[2])
        b = np.sqrt(w[1])
        c = np.sqrt(w[0])
        L = 2*a
        E = 1 - b/a
        P = 1 - c/b
        return L,E,P

In [ ]:
def polyhedronProperties(listSat,list_index):
    """
    Computes L,E and P for all time

    Parameters
    ----------
    list_sat : list
        List of the positions of each satellite, over time.
    listIndex : list
        List of the indexes of the studied satellites.

    Returns
    -------
    L : Array of size (tMax,)
        Characteristic size of the tetrahedron, over time.
    E : Array of size (tMax,)
        Elongation of the tetrahedron, over time.
    P : Array of size (tMax,)
        Planarity of the tetrahedron, over time.

    """
    if len(list_index) ==4:
        max_time_index = len(satellites[0][...])
        L = np.zeros(max_time_index)
        E = np.zeros(max_time_index)
        P = np.zeros(max_time_index)
        for t in range(max_time_index):
            L[t],E[t],P[t] = polyhedronProperties_t(list_sat,list_index,time_index)
        return L,E,P

In [ ]:
def tetra_duo_big_volume_t(poly_list,combination):
    vol = []
    for x in poly_list:
        a = x[0]/2
        b = a*(1-x[1])
        c = b*(1-x[2])
        vol.append((8/3)*a*b*c)
    
    vol_sort = sorted(vol, reverse=True)

    # Extraction of the two biggest tetrahedra volumes and the indexes of their summits
    
    max_tetra = vol_sort[:2]
    max_indexes = [vol.index(tetra) for tetra in max_tetra]
    
    return combination[max_indexes[0]],combination[max_indexes[1]]

In [ ]:
def tetra_most_regular(poly_list,combination):
    
    min_D = float('inf')  
    tetra_regular = None
    index_tetra_regular = None 

    for index, tuple in enumerate(poly_list):
        L,E,P = tuple
        D = m.sqrt(E**2 + P**2)
        if  D < min_D : 
            min_D = D
            tetra_regular = tuple
            index_tetra_regular = index

    if tetra_regular is not None and index_tetra_regular is not None:
        del poly_list[index_tetra_regular]

    #print(tetra_regular)
    return combination[index_tetra_regular]

In [ ]:
def display_helioswarm_satellites_snapshot(index, satellites):
    fig = go.Figure()

    x = [satellites[0][index][0], satellites[1][index][0], satellites[2][index][0], satellites[3][index][0], 
         satellites[4][index][0], satellites[5][index][0], satellites[6][index][0], satellites[7][index][0], 
         satellites[8][index][0]]
    y = [satellites[0][index][1], satellites[1][index][1], satellites[2][index][1], satellites[3][index][1], 
         satellites[4][index][1], satellites[5][index][1], satellites[6][index][1], satellites[7][index][1], 
         satellites[8][index][1]]
    z = [satellites[0][index][2], satellites[1][index][2], satellites[2][index][2], satellites[3][index][2], 
         satellites[4][index][2], satellites[5][index][2], satellites[6][index][2], satellites[7][index][2], 
         satellites[8][index][2]]

    labels = ['HUB', 'SAT_1', 'SAT_2', 'SAT_3', 'SAT_4', 'SAT_5', 'SAT_6', 'SAT_7', 'SAT_8']
    text = [f"{label} (x: {x[i]}, y: {y[i]}, z: {z[i]})" for i, label in enumerate(labels)]

    for i, label in enumerate(labels):
        if label == 'HUB':
            marker_symbol = 'diamond'  
            marker_color = 'blue'  
        else:
            marker_symbol = 'circle'  
            marker_color = 'red' 

        fig.add_trace(go.Scatter3d(
            x=[x[i]],y=[y[i]],z=[z[i]],
            mode='markers',marker=dict(symbol=marker_symbol,  color=marker_color,  size=5,),
            name=label,
            text=[text[i]],
            hoverinfo='text'
        ))

    hub_index = labels.index('HUB')
    position_offset = 1500

    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[x[hub_index] - position_offset, x[hub_index] + position_offset]),
            yaxis=dict(range=[y[hub_index] - position_offset, y[hub_index] + position_offset]),
            zaxis=dict(range=[z[hub_index] - position_offset, z[hub_index] + position_offset]),
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )

    fig.show()

In [ ]:
def display_helioswarm_satellites_tetrahedra(index, satellites,tetra):
    fig = go.Figure()

    x = [satellites[0][index][0], satellites[1][index][0], satellites[2][index][0], satellites[3][index][0], 
         satellites[4][index][0], satellites[5][index][0], satellites[6][index][0], satellites[7][index][0], 
         satellites[8][index][0]]
    y = [satellites[0][index][1], satellites[1][index][1], satellites[2][index][1], satellites[3][index][1], 
         satellites[4][index][1], satellites[5][index][1], satellites[6][index][1], satellites[7][index][1], 
         satellites[8][index][1]]
    z = [satellites[0][index][2], satellites[1][index][2], satellites[2][index][2], satellites[3][index][2], 
         satellites[4][index][2], satellites[5][index][2], satellites[6][index][2], satellites[7][index][2], 
         satellites[8][index][2]]

    labels = ['HUB', 'SAT_1', 'SAT_2', 'SAT_3', 'SAT_4', 'SAT_5', 'SAT_6', 'SAT_7', 'SAT_8']
    text = [f"{label} (x: {x[i]}, y: {y[i]}, z: {z[i]})" for i, label in enumerate(labels)]

    for i, label in enumerate(labels):
        if label == 'HUB':
            marker_symbol = 'diamond' 
            marker_color = 'red' 
        else:
            marker_symbol = 'diamond'  
            marker_color = 'black'  

        fig.add_trace(go.Scatter3d(
            x=[x[i]],y=[y[i]],z=[z[i]],
            mode='markers',marker=dict(symbol=marker_symbol,color=marker_color,  size=5,),
            name=label,
            text=[text[i]],
            hoverinfo='text'
        ))

    hub_index = labels.index('HUB')

    # Add the first tetrahedron
    fig.add_trace(go.Mesh3d( # X, Y, Z coordinates of the 4 summits of the first tetrahedron 
        x=[x[tetra[0][0]], x[tetra[0][1]], x[tetra[0][2]], x[tetra[0][3]]],  
        y=[y[tetra[0][0]], y[tetra[0][1]], y[tetra[0][2]], y[tetra[0][3]]],   
        z=[z[tetra[0][0]], z[tetra[0][1]], z[tetra[0][2]], z[tetra[0][3]]],  
        i=[0, 0, 0],  # Indexes of the summits composing the faces of the first tetrahedron
        j=[1, 2, 3],
        k=[2, 3, 1],
        opacity=0.4,  
        name = 'first_tetrahedron',
        color='orange'  
    ))
    
    # Add the second tetrahedron 
    fig.add_trace(go.Mesh3d(
        x=[x[tetra[1][0]], x[tetra[1][1]], x[tetra[1][2]], x[tetra[1][3]]],  
        y=[y[tetra[1][0]], y[tetra[1][1]], y[tetra[1][2]], y[tetra[1][3]]],  
        z=[z[tetra[1][0]], z[tetra[1][1]], z[tetra[1][2]], z[tetra[1][3]]], 
        i=[0, 0, 0],  
        j=[1, 2, 3],
        k=[2, 3, 1],
        opacity=0.4,  
        name = 'second_tetrahedron',
        color='green'  
    ))

    position_offset = 1500

    fig.update_layout(
        scene=dict(
            aspectmode='cube',
            xaxis=dict(range=[x[hub_index] - position_offset, x[hub_index] + position_offset]),
            yaxis=dict(range=[y[hub_index] - position_offset, y[hub_index] + position_offset]),
            zaxis=dict(range=[z[hub_index] - position_offset, z[hub_index] + position_offset]),
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )
    
    fig.show()
    

In [ ]:
def main():
    
    satellites = np.empty((9, 8521, 3)) # Creation of the empty 3D table that will contain all the x,y,z 
                                        # coordinates of the 9 satellites over the 8521 time indexes

    for i in range(9):                              # Extraction of the coordinates from the 9 .txt files
        file = "./Data/n" + str(i) + ".txt"
        coordinates = satellites_coordinates(file)
        satellites[i] = coordinates
    
    L = list(range(9))     # Creation of the list of indexes of the 9 satellites                        
    C = combinations(L,4)  # List of all the combinations of 4 elements that can be made in the list L 
    pol_t = []             # Empty list that will contain (L,E,P) tuples of the 126 tetrahedra
    
    time_index = int(input("Please choose a value of time_index between 0 and 8521 : "))
    print('time_index corresponds to time : ' + str(retrieve_time("./Data/n0.txt")[time_index]))
    
    question = input("Want another date ? : Yes or No ?")
    while question == "Yes":
        time_index = int(input("Please choose a value of time_index between 0 and 8521 : "))
        print('time_index corresponds to time : ' + str(retrieve_time("./Data/n0.txt")[time_index]))
        question = input("Want another date ? : Yes or No ?")
            
    for comb in C :    
        pol_t.append(polyhedronProperties_t(satellites,comb,time_index))
    
    function_choice = input("Please choose a function that rather : Computes the two tetrahedra that cover the biggest volume (1) or the two tetrahedra that are the most regular (2)")

    # Executes the given function depending on the choice of the user
    if function_choice == '1':
        display_helioswarm_satellites_tetrahedra(time_index, satellites,tetra_duo_big_volume_t(pol_t,C))
    elif function_choice == '2':
        tetra_duo_most_regular = [tetra_most_regular(pol_t,C),tetra_most_regular(pol_t,C)]
        display_helioswarm_satellites_tetrahedra(time_index, satellites,tetra_duo_most_regular)
    else:
        print("Invalid choice. Please choose 1 or 2.")


In [ ]:
main()

In [ ]:
satellites = np.empty((9, 8521, 3)) 
                                        
for i in range(9):                              
    file = "./Data/n" + str(i) + ".txt"
    coordinates = satellites_coordinates(file)
    satellites[i] = coordinates

L = list(range(9))                         
C = combinations(L,4)  
elongation = []
planarity = []
lenght = []

for time_index in range (8521):
    for comb in C :    
        lenght.append(polyhedronProperties_t(satellites,comb,time_index)[0])
        elongation.append(polyhedronProperties_t(satellites,comb,time_index)[1])
        planarity.append(polyhedronProperties_t(satellites,comb,time_index)[2])
        
        


In [ ]:
import plotly.express as px
import plotly.express as px
df = px.data.tips()

fig = px.density_heatmap(x=elongation, y=planarity,title = 'Density heatmap of distribution of tetrahedra',labels = {'x': 'Elongation', 'y': 'Planarity'})

fig.show()


In [ ]:
satellites = np.empty((9, 8521, 3)) # Creation of the empty 3D table that will contain all the x,y,z 
                                        # coordinates of the 9 satellites over the 8521 time indexes

for i in range(9):                              # Extraction of the coordinates from the 9 .txt files
    file = "./Data/n" + str(i) + ".txt"
    coordinates = satellites_coordinates(file)
    satellites[i] = coordinates

print(satellites)